In [13]:
import urllib
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
import time
import threading
from tkinter import*
from PIL import Image, ImageTk
import tkinter.font
from pushbullet import Pushbullet 

In [14]:
driver = webdriver.Chrome('C:/Users/이지민/Desktop/빅데-지정/chromedriver.exe')
driver.implicitly_wait(3) #잠시 대기

In [15]:
class Gui:

    def __init__(self): 
        self.window = Tk()
        self.window.title("실시간 검색어 비교")
        self.window.geometry("1000x800")
        self.window.configure(bg = "skyblue")
        self.font = tkinter.font.Font(family = '210 맨발의청춘 R', size = 23)
        self.font2 = tkinter.font.Font(family = '210 맨발의청춘 R', size = 13)
        self.font3 = tkinter.font.Font(family = '210 맨발의청춘 R', size = 15)
        self.font4 = tkinter.font.Font(family = '210 맨발의청춘 R', size = 20)
        self.font5 = tkinter.font.Font(family = '210 맨발의청춘 R', size = 10)
        
        naver_logo = PhotoImage(file = 'naver.PNG')
        naver_label = Label(self.window, image = naver_logo, bg = 'skyblue')
        naver_label.place(x = 200 , y = 115, width = 200, height = 50)
        
        daum_logo = PhotoImage(file = 'daum.PNG')
        daum_label = Label(self.window, image = daum_logo, bg = 'skyblue')
        daum_label.place(x = 650 , y = 115, width = 200, height = 50)
        
        title_label = Label(self.window, text = '<대표 포털 사이트의 실시간 검색어 비교>', bg = 'skyblue', font = self.font)
        title_label.place(x = 160, y = 65, width = 700, height = 40)
        
        self.message = ''  #문자 알람 기능
        self.pb = Pushbullet('o.hMRCQwU6kHg4QaRSW5KjPWg3K3GP6tlN')
        
        t = threading.Thread(target=self.start_thread)
        t.start()

        self.window.mainloop()

        
    def start_thread(self):
        self.datetime_show()

        self.get_naver_ranking()
        self.put_naver_ranking()

        self.get_daum_ranking()
        self.put_daum_ranking()

        self.get_diif_ranking()

        while True:
            t = 5

            while True:
                timer = Label(self.window, text="갱신 "+ str(t)+ "초 전", bg = 'skyblue', font = self.font5)
                timer.place(x = 5, y = 40, width = 150, height = 30)
                time.sleep(1)
                t = t - 1

                if t == 0:
                    timer = Label(self.window, text="갱신 중", bg = 'skyblue', font = self.font5)
                    timer.place(x = 5, y = 50, width = 150, height = 30)
                    
                    self.datetime_show()
                    
                    self.get_naver_ranking()
                    self.put_naver_ranking()

                    self.get_daum_ranking()
                    self.put_daum_ranking()

                    self.get_diif_ranking()
                    
                    break
                    
        
    def datetime_show(self):
        # 현재 시각을 출력
        now = datetime.now()
        today_label = Label(self.window, text = str(now.year)+"년 "+str(now.month)+"월 "+str(now.day)+"일 "+str(now.hour)+"시 "+str(now.minute)+"분 ", bg = 'skyblue', font = self.font2)
        today_label.place(x = 5, y = 5, width = 250, height = 50)
      
    
    def get_naver_ranking(self):
        self.naver_ranking =[]
        driver.get('https://datalab.naver.com/keyword/realtimeList.naver')
        html = driver.page_source

        soup = BeautifulSoup(html, 'lxml')  #현재 네이버 페이지의 소스를 긁어온다.

        for i in range(10):  #급상승검색어 10위까지만 불러온다.
            title = soup.find_all('span','item_title')
            self.naver_ranking.append(title[i].text)

            
    def put_naver_ranking(self):

        for i in range(10):
            naver_ranking_label = Label(self.window, text = str(i+1) + "위 " +self.naver_ranking[i], bg = 'white', font = self.font3)
            naver_ranking_label.place(x = 150 , y = 170 + (i*40), width = 300, height = 38)
            
    
    def get_daum_ranking(self):
        self.daum_ranking =[]
        driver.get('https://www.daum.net/')
        html = driver.page_source

        soup = BeautifulSoup(html, 'lxml')  #현재 다음 페이지의 소스를 긁어온다.

        for ranking in soup.find(class_='realtime_part').find_all('li'): #class가 realtime_part로 시작하는 것을 찾고 그 중 li로 시작하는 class를 모두 찾는다.
            title = ranking.find(class_='link_issue') #그 중에서도 클래스가 link_issue로 시작하는 것을 찾는다.
            self.daum_ranking.append(title.text)

 
    def put_daum_ranking(self):

        for i in range(10):
            daum_ranking_label = Label(self.window, text = str(i+1) + "위 " +self.daum_ranking[i], bg = 'white', font = self.font3)
            daum_ranking_label.place(x = 600 , y = 170 + (i*40), width = 300, height = 38)
            
            
    def get_diif_ranking(self):
        diff_ranking ={}
        cal = 0
        self.message = ''

        for keyword in self.naver_ranking:
            if keyword in self.daum_ranking: #공통된 keyword가 있다면
                diff = self.naver_ranking.index(keyword) - self.daum_ranking.index(keyword) # 순위 차이 구하기
                diff_ranking[keyword] = abs(int(diff)) #절댓값 씌우기
                cal = cal + (abs(int(diff))*0.001)
                
        if len(diff_ranking) == 0:
            diff_ranking_label = Label(self.window,  bg = 'skyblue',font = self.font2, text = '공통된 검색어가 없습니다.')
            diff_ranking_label.place(x = 10 , y = 600, width = 1000 , height = 30)
            corr = 0
            corr_label = Label(self.window, bg = 'skyblue',font = self.font4, text = '상관계수: ' + str(corr))
            corr_label.place(x = 730, y = 20, width = 300, height = 30)
            push = self.pb.push_note('실시간 포털 사이트 공통 키워드가 없습니다')
            
        else: 
            for i in range(len(diff_ranking)):
                self.message += '▶ ' + str(list(diff_ranking.keys())[i]) + '\n'
                
            push = self.pb.push_note('<실시간 포털 사이트 공통 HOT 키워드>', self.message)  #포털 사이트 공통 키워드 문자 알람 보내기
            
            corr = (len(diff_ranking)*0.1) - cal
            corr_label = Label(self.window, bg = 'skyblue', font = self.font4, text = '상관계수: ' + str(round(corr,4)))
            corr_label.place(x = 730, y = 20, width = 300, height = 30)
            
            diff_ranking_label = Label(self.window, bg = 'skyblue', font = self.font3, text = '{공통 검색어:순위차이} : '+ str(diff_ranking)) #{'공통 검색어' : 순위차이} 형식으로 나타내기
            diff_ranking_label.place(x = 10 , y = 600, width = 1000 , height = 30)
            
            max_value =max(diff_ranking.values())
            min_value =min(diff_ranking.values())
              
            max_keyword = []   #최대, 최소 순위 차이가 나는 검색어가 여러개일 경우를 위한 리스트 
            min_keyword = []
            
            for key,val in diff_ranking.items():
                if max_value == val:
                    max_keyword.append(key)
                    
            for key, val in diff_ranking.items():
                if min_value == val:
                    min_keyword.append(key)
                    
            max_key = ""
            min_key = ""
            
            for row in max_keyword:
                max_key = max_key + " " + row
            for row in min_keyword:
                min_key = min_key + " " + row

            
            max_ranking_label = Label(self.window, bg = 'skyblue',font = self.font4, text = '순위차이가 가장 큰 검색어: '+ max_key + ' (순위차이:' + str(max_value)+')')
            max_ranking_label.place(x = 10 , y = 650, width = 1000, height = 30)
                    
            
            min_ranking_label = Label(self.window, bg = 'skyblue',font = self.font4, text = '순위차이가 가장 작은 검색어: '+ min_key +' (순위차이:' + str(min_value)+')')
            min_ranking_label.place(x = 10 , y = 700, width = 1000, height = 30)
            

In [ ]:
Gui()